Por cada repetición y gesto hacer un cáculo (calcular el valor minimo, el maximo, disviación tipica…). Como resultado: un dataframe con 1 fila x repetición

In [1]:
# importar librerias necesarias
import pandas as pd # para manejar dataframes

In [2]:
# dataframe de los ángulos
# dataframe con toda la informacion recompilada de RawData
df_angle = pd.read_csv('../csvFiles/angles.csv', dtype=object) # salida de leer_dataset.ipynb
# pasar variable obj to numeric
df_angle = df_angle.apply(pd.to_numeric, errors='ignore')
df_angle # visualizacion de df

C:\Users\nerea\AppData\Local\Temp\ipykernel_18548\2558508535.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_angle = df_angle.apply(pd.to_numeric, errors='ignore')


,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,ElbowAngleLeft,ElbowAngleRight,ShoulderAngleLeft,ShoulderAngleRight,WristAngleLeft,WristAngleRight,HipAngleLeft,KneeAngleLeft,AnkleAngleLeft,HipAngleRight,KneeAngleRight,AnkleAngleRight,LeftArmAngle,RightArmAngle,ArmsTogetherAngle
0,101,0,1,1,stand,18.731844,12.815119,142.560727,140.857143,2.598764,6.344406,104.574768,3.743551,64.246261,101.521850,4.855627,60.940986,156.791112,159.877408,152.580270
1,101,0,1,1,stand,18.682011,13.214648,142.757760,140.967721,2.522847,5.905771,104.499195,3.821802,64.334729,101.590425,4.885456,61.134582,156.667813,160.035203,152.602527
2,101,0,1,1,stand,18.530184,14.175895,142.724795,140.886094,3.239981,6.639703,104.558119,3.832570,64.078901,101.492094,4.776421,59.972082,156.747131,160.251764,152.596513
3,101,0,1,1,stand,18.525441,15.675640,142.796223,140.996819,3.488477,10.172814,104.502499,3.642046,63.451166,101.422947,4.655747,59.366231,156.715615,160.589502,152.653362
4,101,0,1,1,stand,18.440644,18.672491,142.755019,140.986030,3.640028,9.700975,104.554046,3.670732,63.212740,101.311001,4.601472,59.328043,156.793484,160.668306,152.675640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228296,307,8,9,1,stand,19.235168,16.366695,142.466122,142.949282,10.597409,8.590306,107.710008,17.152761,57.808079,94.948037,1.709497,41.616584,156.103380,158.651430,149.864559
228297,307,8,9,1,stand,19.337504,16.460773,142.614506,142.959204,9.097187,8.083043,107.352914,16.479229,57.733793,94.653231,1.263963,41.879291,156.056261,158.714378,149.734213
228298,307,8,9,1,stand,19.762179,16.292120,142.579738,142.903745,9.288112,10.157666,107.398951,16.368690,58.670153,94.346427,1.041109,78.704138,156.130164,158.836560,149.712017
228299,307,8,9,1,stand,20.339779,16.195526,142.685324,142.897308,7.335038,9.108455,107.436579,16.163032,57.798046,94.144821,1.015447,78.752737,156.114139,158.911807,149.496620


In [3]:
groups = df_angle.groupby(["SubjectID", "GestureLabel", "RepetitionNumber"])
data = []
for (subject_id, gesture_label, repetition_number), group in groups:
    angles = group.iloc[:, 5:] # seleccionamos las columnas de los angulos
    means = angles.mean()
    std_devs = angles.std()

    data.append({
        'SubjectID': subject_id,
        'GestureLabel': gesture_label,
        'RepetitionNumber': repetition_number,
        'CorrectLabel': group['CorrectLabel'].iloc[0],
        'Position': group['Position'].iloc[0],
        'Duration': len(group),
        'standardDeviation': std_devs,
        'Maximum': angles.max(),
        'Minimum': angles.min(),
        'Mean': means,
        'Range': angles.max() - angles.min(),
        'Variance': angles.var(),
        'CoV': std_devs / means, # coefficient of variation
        'Skewness': angles.skew(),  # Skewness
        'Kurtosis': angles.kurtosis()  # Kurtosis
    })




In [4]:
df = pd.DataFrame(data)
df = df.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

In [5]:
def extract_keys_and_create_columns(column_name):
    keys = set().union(*(d.keys() for d in df[column_name]))
    data = {}
    for key in keys:
        data[key + '_' + column_name] = df[column_name].apply(lambda x: x.get(key))
    return pd.DataFrame(data)

# Extract keys and create new columns for each dictionary column
new_columns = pd.concat([extract_keys_and_create_columns(col) for col in ['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range', 'Variance', 'CoV', 'Skewness', 'Kurtosis']], axis=1)

# Concatenate new columns to the DataFrame
df = pd.concat([df, new_columns], axis=1)

# Drop the original dictionary columns
df = df.drop(['standardDeviation', 'Maximum', 'Minimum', 'Mean', 'Range', 'Variance', 'CoV', 'Skewness', 'Kurtosis'], axis=1)

# Sort DataFrame by specified columns
df = df.sort_values(['SubjectID', 'GestureLabel', 'RepetitionNumber'])

In [7]:
df

,SubjectID,GestureLabel,RepetitionNumber,CorrectLabel,Position,Duration,HipAngleLeft_standardDeviation,RightArmAngle_standardDeviation,KneeAngleLeft_standardDeviation,AnkleAngleLeft_standardDeviation,...,ElbowAngleRight_Kurtosis,LeftArmAngle_Kurtosis,ShoulderAngleRight_Kurtosis,WristAngleLeft_Kurtosis,ElbowAngleLeft_Kurtosis,WristAngleRight_Kurtosis,ShoulderAngleLeft_Kurtosis,HipAngleRight_Kurtosis,ArmsTogetherAngle_Kurtosis,AnkleAngleRight_Kurtosis
0,101,0,1,1,stand,59,0.360868,2.765557,0.208706,0.658147,...,-1.509605,0.537440,0.247256,-1.154852,-0.917690,0.198533,-0.635142,-0.924047,-0.604303,-0.404407
1,101,0,2,1,stand,44,0.118578,2.298280,0.221489,0.831351,...,-1.280617,0.726806,-1.751783,-0.770170,0.418772,1.048145,-0.903003,-0.198608,0.893421,-0.276498
2,101,0,3,1,stand,53,0.246191,3.445304,0.200163,0.960267,...,-1.455247,0.140506,0.719889,-0.580654,-0.761280,-0.393123,0.412761,-1.007602,-0.413869,-0.530532
3,101,0,4,1,stand,57,0.283214,3.626022,0.344905,0.573168,...,-1.514596,1.060234,0.495865,-0.136403,-0.584491,-0.610934,-1.330692,-1.487544,-1.258808,-0.057357
4,101,0,5,1,stand,60,0.393636,3.767307,0.217658,0.658998,...,-1.360146,-1.527706,0.393907,-0.426884,-0.534952,-1.170145,-0.844501,-1.427344,-0.338747,-0.322273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,307,8,8,1,stand,48,8.563372,1.555034,6.503301,10.353342,...,0.234936,-0.781945,0.465005,-0.949755,-0.872651,0.056064,-0.971195,-1.014746,-0.339300,-0.270964
2585,307,8,9,1,stand,48,7.609432,1.332626,6.647659,4.728600,...,-0.759991,-1.644095,-0.019000,0.520658,-1.114747,-1.006323,-0.963412,-1.651462,-0.863515,-1.671159
2586,307,8,10,1,stand,47,6.656025,1.315829,5.285898,8.033854,...,-0.707586,-0.826005,-1.203659,-0.834273,-0.267076,-0.776532,0.037584,-1.103517,-0.449516,-2.061997
2587,307,8,11,1,stand,51,7.627692,1.896625,6.237513,9.185235,...,-0.899253,-0.620023,-0.087088,-0.261026,-0.983178,-0.052683,-0.660478,-1.697380,-0.445218,-1.011064


In [8]:
df.to_csv('../csvFiles/medidasPerRepetition.csv', index=False)